In [21]:
import numpy as np

from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [1]:
import json, re, ast, numpy as np, scipy.sparse as sp
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k

# Загружаем MovieLens 100k
movielens = fetch_movielens()
train, test = movielens['train'], movielens['test']
item_labels = movielens['item_labels']          # Сопоставление item_id → название фильма

# ---------- 1.1 Парсим taxonomy-файл ----------
taxonomy_path = '/Users/icereal/Desktop/vsc_projects/ml-100k/movie_taxonomy_10feats_temp.json'
with open(taxonomy_path, 'r', encoding='utf-8') as f:
    raw = json.load(f)

def parse_feature_blob(blob: str) -> dict:
    """Убираем ```json … ``` / одинарные кавычки и превращаем в словарь."""
    clean = re.sub(r'```json|```', '', blob, flags=re.S).strip()
    try:
        return json.loads(clean)                # большинство блоков
    except json.JSONDecodeError:
        return ast.literal_eval(clean)          # в отдельных записях используются одинарные кавычки

taxonomy = {title: parse_feature_blob(blob) for title, blob in raw.items()}


In [ ]:
# ---------- 2.1 Собираем токены ----------
item_tokens = {}
all_tokens = set()

for item_id, title in enumerate(item_labels):
    meta = taxonomy.get(title, {})              # если фильма нет в taxonomy – получим {}
    tokens = []
    for k, v in meta.items():
        if v and v != 'Unknown':
            tokens.append(f'{k}:{v}')
    item_tokens[item_id] = tokens
    all_tokens.update(tokens)

# ---------- 2.2 Строим Dataset ----------
n_users, n_items = train.shape
ds = Dataset()
ds.fit(users=np.arange(n_users),
       items=np.arange(n_items),
       item_features=all_tokens)

# ---------- 2.3 Конвертируем взаимодействия ----------
def to_interactions(mat):
    coo = mat.tocoo()
    return ds.build_interactions(zip(coo.row, coo.col))[0]   # weights нам не нужны

train_int = to_interactions(train)
test_int = to_interactions(test)

# ---------- 2.4 Матрица item-features ----------
item_features = ds.build_item_features(
    [(i, toks) for i, toks in item_tokens.items()])


In [ ]:
def ndcg_at_k(model, interactions, train_interactions, k=10, item_features=None):
    """Вычисляет средний nDCG@k по всем пользователям."""
    n_users, n_items = interactions.shape
    ndcgs = []

    for user_id in range(n_users):
        # Пропускаем, если у пользователя нет положительных взаимодействий в тесте
        test_items = interactions.tocsr()[user_id].indices
        if len(test_items) == 0:
            continue

        scores = model.predict(user_id,
                               np.arange(n_items),
                               item_features=item_features,
                               user_features=None)

        # Маскируем обучающие взаимодействия (не рекомендовать уже известное)
        train_items = train_interactions.tocsr()[user_id].indices
        scores[train_items] = -np.inf

        # Получаем ранжированный список
        ranked = np.argsort(-scores)[:k]

        # Relevance — бинарная: 1, если item в тесте, иначе 0
        gains = np.isin(ranked, test_items).astype(np.float32)

        # Discounted Cumulative Gain
        discounts = np.log2(np.arange(2, k + 2))
        dcg = (gains / discounts).sum()

        # Ideal DCG (если бы мы идеально угадали)
        ideal_gains = np.sort(gains)[::-1]
        idcg = (ideal_gains / discounts).sum()

        ndcg = dcg / idcg if idcg > 0 else 0.0
        ndcgs.append(ndcg)

    return np.mean(ndcgs)


# Базовая (только взаимодействия)
model_base = LightFM(learning_rate=0.05, loss='warp')
model_base.fit(train_int, epochs=10)

# С предметными фичами
model_feat = LightFM(learning_rate=0.05, loss='warp')
model_feat.fit(train_int, item_features=item_features, epochs=10)

# ---------- 3.1 Метрики ----------
# ---------- 3.1 Метрики ----------
def evaluate(m, name, **kwargs):
    r_test   = recall_at_k(m, test_int, k=10,
                           train_interactions=train_int, **kwargs).mean()
    ndcg_test = ndcg_at_k(m, test_int, k=10,
                          train_interactions=train_int, **kwargs).mean()
    print(f'{name:^12} | R@10 test={r_test:.3f} | nDCG@10 test={ndcg_test:.3f}')



evaluate(model_base, 'baseline')
evaluate(model_feat, 'with feats', item_features=item_features)


# Other